<a href="https://colab.research.google.com/github/youse0ng/AIProgramming/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D5%EC%A3%BC%EC%B0%A8_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10 # cifar10 데이터셋
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import numpy as np
import os
from tensorflow.keras import utils

In [ ]:
try:
  os.mkdir('my_log_dirs8')  # my_log_dirs8 이라는 폴더생성
  callback=[keras.callbacks.TensorBoard(log_dir='my_log_dirs8',histogram_freq=1,embeddings_freq=1)]
except:
  print("폴더가 이미 있습니다.")

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
y_train,y_train.shape

In [ ]:
x_train[0]

In [ ]:
x_train.shape

In [ ]:
x_train[0].shape

In [ ]:
y_train.shape

In [ ]:
width = 32
height = 32
channel = 3
# 순차적 모델을 만들기 위한 Sequential() 함수
model = Sequential(name='CIFAR10_CNN') # 6개의 Conv2d layer
# filters : Kernel 그룹의 갯수 / kernel_size : Kernel 크기
model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu',
                 input_shape=(width, height, channel)))
model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
# pool_size : MaxPooling 의 Window size
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
# N차원 배열을 1차원으로 펴줍니다
model.add(layers.Flatten())
# 출력층 MLP
model.add(layers.Dense(10, activation='softmax')) # 10개의 node의 값을 -> softmax 함수로 통해 확률로 바꿈. 0~9까지의 클래스 확률로 나뉘어지고 제일 높은 값의 인덱스가 클래스넘버가 된다.
model.summary()

In [ ]:
x_train = x_train.astype('float32') / 255.0   # x_train의 화소값 (np.array)로 될 것이니까... astype으로 데이터 자료형을 float32로 바꾸고 normalization
x_test = x_test.astype('float32') / 255.0 # 0~1사이의 화소값

y_train = utils.to_categorical(y_train) # (이게 아마... 범주형 카테고리를 기계가 이해할 수 있는 원핫 인코딩하는 거였나 기억이 잘안난다.)
y_test = utils.to_categorical(y_test)

In [ ]:
y_train.shape

In [ ]:
x_train[1]

In [ ]:
class myCallback(keras.callbacks.Callback): # keras.callbacks 모듈에 있는 Callback이라는 클래스를 상속
  def on_epoch_end(self, epoch, logs={}): # 상속하고 def __init__을 안해도 자연스럽게 Callback의 def __init__ 을 사용할 수 있다.
    if logs.get('acc')>0.97:
      print("Cancel the training")
      self.model.stop_training=True # def __init__ 에 있는 self.model.stop_training에 접근

callbacks=myCallback() # callbacks 인스턴스 = 클래스 (선언) 그러면 callbacks.on_epoch_end(epoch) 로 메소드를 사용할 수 있다.

In [ ]:
callbacks=[keras.callbacks.TensorBoard(
    log_dir='my_log_dir7',
    histogram_freq=1,
    embeddings_freq=1,
)
]

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #

In [ ]:
hist = model.fit(x_train, y_train,
                 epochs=5,
                 batch_size=16,
                 validation_data=(x_test, y_test),callbacks=callbacks)

Epoch 15: train_정확도 90% # val_정확도:0.7342 이 가장 무난하다.

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history['loss'], 'y', label='train loss')
plt.plot(hist.history['val_loss'], 'r', label='val loss')
plt.ylim([0.0, 2.5])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(hist.history['accuracy'], 'y', label='train acc')
plt.plot(hist.history['val_accuracy'], 'r', label='val acc')
plt.xlabel('Epochs')
plt.ylabel('acc')
plt.legend()
plt.show()

이런 경우에는 train_acc는 높고, val_acc는 상대적으로 낮다.

train 데이터셋을 학습을 너무 잘해서 나타나는 결과인 overfitting 과적합 현상이라고 생각된다.

과적합을 방지할 방법 중 모델에 적용할 기법은

* 배치 정규화
* 드롭아웃
* 얼리스탑핑

데이터셋에 적용할 기법
* Data Augmentation

을 다음에 적용해보자

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=my_log_dirs8